In [227]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

!pip install koreanize-matplotlib

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' ## 그림 더 선명하게

In [228]:
non_cov=pd.read_csv('코로나아닌공연clust.csv', low_memory=False)

In [229]:
def count_price(row):
    return row[['price1', 'price2', 'price3', 'price4', 'price5', 'price6']].count()
non_cov['count_price']=non_cov.apply(count_price, axis=1)

In [230]:
df=non_cov.copy()
df.groupby('count_price').label.count()

count_price
1      1437
2     14120
3     27704
4    106742
5     60040
Name: label, dtype: int64

## 코로나 아닌 공연

### 좌석 1개

In [231]:
result_list=[]
for cl in df.cluster.unique():
    test1=df[df.cluster==cl]
    test=test1[test1.count_price==1]
    max_total = -np.inf
    best_price1 = 0
    best_price1_per = 0
    best_price1_len = 0
    if len(test)>5:
        for price1 in range(int(test['price1'].min()), int(test['price1'].mean()+30000), 2500):
            df_copy = test.copy()
            df_copy['price1'] = price1

            # 조건에 따라 afford와 total_price 비교하여 필터링
            price1_sum = df_copy[df_copy['afford'] >= 0]['price'].sum()
            # 필터링된 데이터에서 total_price 합계 계산
            total = price1_sum*price1/int(test['price1'].mean())
            
            # 최대 총합 및 price1, price2 값 갱신
            if total > max_total:
                max_total = total
                best_price1 = price1
                best_price1_per = price1/int(test['price1'].mean())
                best_price1_len=len(df_copy[df_copy['afford'] >= df_copy['price1'].min()])
    result_list.append({'cluster': cl, 
                        'best_price1': best_price1,
                        'best_price1_per': best_price1_per,
                        'best_price1_len': best_price1_len,
                        'max_total': max_total-test['price'].sum()})
result_df1 = pd.DataFrame(result_list)

In [232]:
result_df1

,cluster,best_price1,best_price1_per,best_price1_len,max_total
0,0,0,0.00,0,-inf
1,1,0,0.00,0,-inf
2,2,0,0.00,0,-inf
3,3,37500,3.75,0,36272500.0


In [233]:
non_cov.loc[(non_cov.cluster==3)&(non_cov.count_price==1), 'price1']=non_cov.loc[(non_cov.cluster==3)&(non_cov.count_price==1), 'price1']*result_df1.loc[3, 'best_price1_per']

### 좌석 2개 - 3개 공연 폐기

In [234]:
result_list=[]
for cl in df.cluster.unique():
    test1=df[df.cluster==cl]
    test=test1[test1.count_price==2]
    max_total = -np.inf
    best_price1 = 0
    best_price2 = 0
    best_price1_per = 0
    best_price2_per = 0
    best_price1_len = 0
    best_price2_len = 0
    if test.empty==False:
        for price1 in range(int(test['price2'].mean()), int(test['price1'].mean()+10000), 10000):
            for price2 in range(int(test['price2'].mean()), int(price1), 10000):
                df_copy = test.copy()
                df_copy['price1'] = price1
                df_copy['price2'] = price2
                if any((test.cluster==0)|(test.cluster==2)):
                    # 조건에 따라 afford와 total_price 비교하여 필터링
                    price1_sum = df_copy[df_copy['afford'] >= (df_copy['price1']-df_copy['price2']+50000)]['price'].sum()
                    price2_sum = df_copy[(df_copy['afford'] < (df_copy['price1']-df_copy['price2']+50000))]['price'].sum()
                else:
                    # 조건에 따라 afford와 total_price 비교하여 필터링
                    price1_sum = df_copy[df_copy['afford'] >= (df_copy['price1']-df_copy['price2']+25000)]['price'].sum()
                    price2_sum = df_copy[(df_copy['afford'] < (df_copy['price1']-df_copy['price2']+25000))]['price'].sum()
                # 필터링된 데이터에서 total_price 합계 계산
                total = price1_sum*price1/test['price1'].mean()+price2_sum*price2/test['price2'].mean()
                # 최대 총합 및 price1, price2 값 갱신
                if total > max_total:
                    max_total = total
                    best_price1 = price1
                    best_price2 = price2
                    best_price1_per = price1/int(test['price1'].mean())
                    best_price2_per = price2/int(test['price2'].mean())
                    best_price1_len=len(df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price2']])
                    best_price2_len=len(df_copy[(df_copy['afford'] < df_copy['price1']-df_copy['price2'])])
    result_list.append({'cluster': cl, 
                        'best_price1': best_price1, 'best_price2': best_price2,
                        'best_price1_per': best_price1_per, 'best_price2_per': best_price2_per,
                        'best_price1_len': best_price1_len, 'best_price2_len': best_price2_len,
                       'max_total': max_total-test['price'].sum()})            
result_df2 = pd.DataFrame(result_list)

In [235]:
result_df2

,cluster,best_price1,best_price2,best_price1_per,best_price2_per,best_price1_len,best_price2_len,max_total
0,0,58623,48623,1.105344,1.698739,88,148,5.405311e+06
1,1,35024,25024,1.166223,1.665602,7166,4307,1.671407e+08
2,2,50000,40000,1.000000,1.333333,115,72,1.440333e+06
3,3,31099,21099,1.000000,1.000000,303,1202,-4.725709e+06


In [236]:
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==2), 'price1']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==2), 'price1']*result_df2.loc[i, 'best_price1_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==2), 'price2']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==2), 'price2']*result_df2.loc[i, 'best_price2_per']

### 좌석 3개

In [237]:
result_list=[]
for cl in df.cluster.unique():
    test1=df[df.cluster==cl]
    test=test1[test1.count_price==3]
    max_total = -np.inf
    best_price1 = 0
    best_price2 = 0
    best_price3 = 0
    best_price1_per = 0
    best_price2_per = 0
    best_price3_per = 0
    best_price1_len = 0
    best_price2_len = 0
    best_price3_len = 0
    if test.empty==False:
        for price1 in range(int(test['price2'].mean()), int(test['price1'].mean()+20000), 10000):
            for price2 in range(int(test['price3'].mean()), int(price1), 10000):
                for price3 in range(int(test['price3'].mean()), int(price2), 10000):
                    df_copy = test.copy()
                    df_copy['price1'] = price1
                    df_copy['price2'] = price2
                    df_copy['price3'] = price3

                    # 조건에 따라 afford와 total_price 비교하여 필터링
                    if any((test.cluster==0)|(test.cluster==2)):
                        price1_sum = df_copy[df_copy['afford'] >= (df_copy['price1']-df_copy['price3']+12000)]['price'].sum()
                        price2_sum = df_copy[(df_copy['afford'] >= (df_copy['price2']-df_copy['price3']+12000))
                                             &(df_copy['afford'] < (df_copy['price1']-df_copy['price3']))+12000]['price'].sum()
                        price3_sum = df_copy[(df_copy['afford'] < (df_copy['price2']-df_copy['price3'])+12000)]['price'].sum()
                    else:
                        price1_sum = df_copy[df_copy['afford'] >= (df_copy['price1']-df_copy['price3'])]['price'].sum()
                        price2_sum = df_copy[(df_copy['afford'] >= (df_copy['price2']-df_copy['price3']))
                                             &(df_copy['afford'] < (df_copy['price1']-df_copy['price3']))]['price'].sum()
                        price3_sum = df_copy[(df_copy['afford'] < (df_copy['price2']-df_copy['price3']))]['price'].sum()
                    # 필터링된 데이터에서 total_price 합계 계산
                    total = price1_sum*price1/test['price1'].mean()+price2_sum*price2/test['price2'].mean()
                    + price3_sum*price3/test['price3'].mean()

                    # 최대 총합 및 price1, price2 값 갱신
                    if total > max_total:
                        max_total = total
                        best_price1 = price1
                        best_price2 = price2
                        best_price3 = price3
                        best_price1_per = price1/int(test['price1'].mean())
                        best_price2_per = price2/int(test['price2'].mean())
                        best_price3_per = price3/int(test['price3'].mean())
                        best_price1_len=len(df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price3']])/len(df_copy[df_copy['afford']>= df_copy['price1'].mean()-df_copy['price3'].mean()])
                        best_price2_len=len(df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price3'])
                                                    &(df_copy['afford'] < df_copy['price1']-df_copy['price3'])])
                        best_price3_len=len(df_copy[(df_copy['afford'] < df_copy['price2']-df_copy['price3'])])
    result_list.append({'cluster': cl, 
                        'best_price1': best_price1, 'best_price2': best_price2,
                        'best_price3': best_price3,
                        'best_price1_per': best_price1_per, 'best_price2_per': best_price2_per,
                        'best_price3_per': best_price3_per,
                        'best_price1_len': best_price1_len, 'best_price2_len': best_price2_len,
                        'best_price3_len': best_price3_len,
                        'max_total': max_total-test['price'].sum()})            
result_df3 = pd.DataFrame(result_list)

In [238]:
result_df3

,cluster,best_price1,best_price2,best_price3,best_price1_per,best_price2_per,best_price3_per,best_price1_len,best_price2_len,best_price3_len,max_total
0,0,64971,39986,29986,1.231141,1.143405,1.500350,1.0,1192,905,-3.585635e+07
1,1,69322,63208,53208,1.219127,1.607446,2.292658,1.0,1269,2180,1.052129e+07
2,2,69573,64573,54573,1.258852,1.631744,2.220852,1.0,2804,4833,-2.305613e+08
3,3,75703,59273,49273,1.217834,1.296917,1.683223,1.0,2909,2541,4.877496e+06


In [239]:
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price1']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price1']*result_df3.loc[i, 'best_price1_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price2']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price2']*result_df3.loc[i, 'best_price2_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price3']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==3), 'price3']*result_df3.loc[i, 'best_price3_per']

### 좌석 4개

In [240]:
result_list=[]
for cl in df.cluster.unique():
    test1=df[df.cluster==cl]
    test=test1[test1.count_price==4]
    max_total = -np.inf
    best_price1 = 0
    best_price2 = 0
    best_price3 = 0
    best_price4 = 0
    best_price1_per = 0
    best_price2_per = 0
    best_price3_per = 0
    best_price4_per = 0
    best_price6_per = 0
    best_price1_len = 0
    best_price2_len = 0
    best_price3_len = 0
    best_price4_len = 0
    if test.empty==False:
        for price1 in range(int(test['price1'].mean()), int(test['price1'].mean()+20000), 10000):
            for price2 in range(int(test['price2'].mean()), int(price1), 10000):
                for price3 in range(int(test['price3'].mean()), int(price2), 10000):
                    for price4 in range(int(test['price4'].mean()), int(price3), 10000):
                        df_copy = test.copy()
                        df_copy['price1'] = price1
                        df_copy['price2'] = price2
                        df_copy['price3'] = price3
                        df_copy['price4'] = price4

                        # 조건에 따라 afford와 total_price 비교하여 필터링
                        if any((test.cluster==0)|(test.cluster==2)):
                            price1_sum = df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price4']+11000]['price'].sum()
                            price2_sum = df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price4']+11000)
                                                 &(df_copy['afford'] < df_copy['price1']-df_copy['price4'])+11000]['price'].sum()
                            price3_sum = df_copy[(df_copy['afford'] >= df_copy['price3']-df_copy['price4']+11000)
                                                 &(df_copy['afford'] < df_copy['price2']-df_copy['price4']+11000)]['price'].sum()
                            price4_sum = df_copy[(df_copy['afford'] < df_copy['price3']-df_copy['price4']+11000)]['price'].sum()
                            # 필터링된 데이터에서 total_price 합계 계산
                            total = price1_sum*price1/test['price1'].mean()+price2_sum*price2/test['price2'].mean()
                            + price3_sum*price3/test['price3'].mean()+price3_sum*price3/test['price4'].mean()
                        else:
                            price1_sum = df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price4']]['price'].sum()
                            price2_sum = df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price4'])
                                                 &(df_copy['afford'] < df_copy['price1']-df_copy['price4'])]['price'].sum()
                            price3_sum = df_copy[(df_copy['afford'] >= df_copy['price3']-df_copy['price4'])
                                                 &(df_copy['afford'] < df_copy['price2']-df_copy['price4'])]['price'].sum()
                            price4_sum = df_copy[(df_copy['afford'] < df_copy['price3']-df_copy['price4'])]['price'].sum()
                            # 필터링된 데이터에서 total_price 합계 계산
                            total = price1_sum*price1/test['price1'].mean()+price2_sum*price2/test['price2'].mean()
                            + price3_sum*price3/test['price3'].mean()+price3_sum*price3/test['price4'].mean()
                        # 최대 총합 및 price1, price2 값 갱신
                        if total > max_total:
                            max_total = total
                            best_price1 = price1
                            best_price2 = price2
                            best_price3 = price3
                            best_price4 = price4
                            best_price1_per = price1/int(test['price1'].mean())
                            best_price2_per = price2/int(test['price2'].mean())
                            best_price3_per = price3/int(test['price3'].mean())
                            best_price4_per = price4/int(test['price4'].mean())
                            best_price1_len=len(df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price4']])
                            best_price2_len=len(df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price4'])
                                                        &(df_copy['afford'] < df_copy['price1']-df_copy['price4'])])
                            best_price3_len=len(df_copy[(df_copy['afford'] >= df_copy['price3']-df_copy['price4'])
                                                        &(df_copy['afford'] < df_copy['price2']-df_copy['price4'])])
                            best_price4_len=len([(df_copy['afford'] < df_copy['price3']-df_copy['price4'])])
    result_list.append({'cluster': cl, 
                        'best_price1': best_price1, 'best_price2': best_price2,
                        'best_price3': best_price3, 'best_price4': best_price4,
                        'best_price1_per': best_price1_per, 'best_price2_per': best_price2_per,
                        'best_price3_per': best_price3_per, 'best_price4_per': best_price4_per,
                        'best_price1_len': best_price1_len, 'best_price2_len': best_price2_len,
                        'best_price3_len': best_price3_len, 'best_price4_len': best_price4_len,
                        'max_total': max_total-test['price'].sum()})            
result_df4 = pd.DataFrame(result_list)

In [241]:
result_df4

,cluster,best_price1,best_price2,best_price3,best_price4,best_price1_per,best_price2_per,best_price3_per,best_price4_per,best_price1_len,best_price2_len,best_price3_len,best_price4_len,max_total
0,0,102650,98771,95054,92079,1.107933,1.436230,2.109779,4.170433,4760,271,33,1,-7.181039e+07
1,1,107181,91966,91213,81307,1.102901,1.277909,1.781052,2.597087,20407,8492,293,1,3.108408e+07
2,2,117277,110771,104155,101784,1.093217,1.371420,1.923276,3.202366,11788,1924,1232,1,-2.836536e+08
3,3,137219,127027,118285,113011,1.078605,1.309192,1.732225,2.627491,22605,5497,1077,1,-1.602816e+08


In [242]:
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price1']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price1']*result_df4.loc[i, 'best_price1_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price2']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price2']*result_df4.loc[i, 'best_price2_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price3']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price3']*result_df4.loc[i, 'best_price3_per']
for i in range(0,4):
    non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price4']=non_cov.loc[(non_cov.cluster==i)&(non_cov.count_price==4), 'price3']*result_df4.loc[i, 'best_price4_per']

In [243]:
non_cov.loc[non_cov.count_price!=5, ['price1']]=round(non_cov.loc[non_cov.count_price!=5, ['price1']],-4)
non_cov.loc[non_cov.count_price!=5, ['price2']]=round(non_cov.loc[non_cov.count_price!=5, ['price2']],-4)
non_cov.loc[non_cov.count_price!=5, ['price3']]=round(non_cov.loc[non_cov.count_price!=5, ['price3']],-4)
non_cov.loc[non_cov.count_price!=5, ['price4']]=round(non_cov.loc[non_cov.count_price!=5, ['price4']],-4)

In [244]:
df.loc[non_cov.count_price!=5, ['price1', 'price2', 'price3', 'price4']].describe()

,price1,price2,price3,price4
count,150003.000000,148566.000000,134446.000000,101390.000000
mean,91932.674680,68705.120956,51026.419529,35308.067857
std,53354.326767,39819.576487,27049.245129,18834.220321
min,10000.000000,10000.000000,10000.000000,10000.000000
25%,50000.000000,40000.000000,30000.000000,10000.000000
50%,90000.000000,70000.000000,50000.000000,40000.000000
75%,120000.000000,90000.000000,70000.000000,50000.000000
max,300000.000000,180000.000000,120000.000000,80000.000000


In [245]:
non_cov.loc[non_cov.count_price!=5, ['price1', 'price2', 'price3', 'price4']].describe()

,price1,price2,price3,price4
count,150003.000000,148566.000000,134446.000000,106742.000000
mean,102784.744305,92180.579675,92829.239992,295831.256675
std,56749.456127,52704.156653,48003.698217,134230.371122
min,20000.000000,20000.000000,20000.000000,90000.000000
25%,60000.000000,50000.000000,50000.000000,180000.000000
50%,100000.000000,90000.000000,100000.000000,310000.000000
75%,130000.000000,120000.000000,120000.000000,370000.000000
max,330000.000000,250000.000000,230000.000000,880000.000000


### 좌석 5개 - 너무 세분화 되어 있어 제대로 알고리즘이 작동하지 않음(폐기)

In [247]:
'''
result_list=[]
for cl in df.cluster.unique():
    test1=df[df.cluster==cl]
    test=test1[test1.count_price==5]
    max_total = -np.inf
    best_price1 = 0
    best_price2 = 0
    best_price3 = 0
    best_price4 = 0
    best_price5 = 0
    best_price1_per = 0
    best_price2_per = 0
    best_price3_per = 0
    best_price4_per = 0
    best_price5_per = 0
    best_price1_len = 0
    best_price2_len = 0
    best_price3_len = 0
    best_price4_len = 0
    best_price5_len = 0
    if test.empty==False:
        for price1 in range(int(test['price1'].mean()-20000), int(test['price1'].mean()+20000), 10000):
            for price2 in range(int(test['price2'].mean()-20000), int(price1), 10000):
                for price3 in range(int(test['price3'].mean()-20000), int(price2), 10000):
                    for price4 in range(int(test['price4'].mean()-20000), int(price3), 10000):
                        for price5 in range(int(test['price5'].mean()-20000), int(price4), 10000):
                            df_copy = test.copy()
                            df_copy['price1'] = price1
                            df_copy['price2'] = price2
                            df_copy['price3'] = price3
                            df_copy['price4'] = price4
                            df_copy['price5'] = price5

                            # 조건에 따라 afford와 total_price 비교하여 필터링
                            price1_sum = df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price5']]['price'].sum()
                            price2_sum = df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price5'])
                                                 &(df_copy['afford'] < df_copy['price1']-df_copy['price5'])]['price'].sum()
                            price3_sum = df_copy[(df_copy['afford'] >= df_copy['price3']-df_copy['price5'])
                                                 &(df_copy['afford'] < df_copy['price2']-df_copy['price5'])]['price'].sum()
                            price4_sum = df_copy[(df_copy['afford'] >= df_copy['price4']-df_copy['price5'])
                                                 &(df_copy['afford'] < df_copy['price3']-df_copy['price5'])]['price'].sum()
                            price5_sum = df_copy[(df_copy['afford'] < df_copy['price4']-df_copy['price5'])]['price'].sum()
                            # 필터링된 데이터에서 total_price 합계 계산
                            total = price1_sum*price1/test['price1'].mean()+price2_sum*price2/test['price2'].mean()
                            + price3_sum*price3/test['price3'].mean()+price4_sum*price4/test['price4'].mean()
                            + price5_sum*price5/test['price5'].mean()

                            # 최대 총합 및 price1, price2 값 갱신
                            if total > max_total:
                                max_total = total
                                best_price1 = price1
                                best_price2 = price2
                                best_price3 = price3
                                best_price4 = price4
                                best_price5 = price5
                                best_price1_per = price1/int(test['price1'].mean())
                                best_price2_per = price2/int(test['price2'].mean())
                                best_price3_per = price3/int(test['price3'].mean())
                                best_price4_per = price4/int(test['price4'].mean())
                                best_price5_per = price5/int(test['price5'].mean())
                                best_price1_len=len(df_copy[df_copy['afford'] >= df_copy['price1']-df_copy['price5']])
                                best_price2_len=len(df_copy[(df_copy['afford'] >= df_copy['price2']-df_copy['price5'])
                                                            &(df_copy['afford'] < df_copy['price1']-df_copy['price5'])])
                                best_price3_len=len(df_copy[(df_copy['afford'] >= df_copy['price3']-df_copy['price5'])
                                                            &(df_copy['afford'] < df_copy['price2']-df_copy['price5'])])
                                best_price4_len=len([(df_copy['afford'] >= df_copy['price4']-df_copy['price5'])
                                                     &(df_copy['afford'] < df_copy['price3']-df_copy['price5'])])
                                best_price5_len=len([(df_copy['afford'] < df_copy['price4']-df_copy['price5'])])
result_list.append({'cluster': cluster, 
                    'best_price1': best_price1, 'best_price2': best_price2,
                    'best_price3': best_price3, 'best_price4': best_price4,
                    'best_price5': best_price5,
                    'best_price1_per': best_price1_per, 'best_price2_per': best_price2_per,
                    'best_price3_per': best_price3_per, 'best_price4_per': best_price4_per, 
                    'best_price5_per': best_price5_per, 
                    'best_price1_len': best_price1_len, 'best_price2_len': best_price2_len,
                    'best_price3_len': best_price3_len, 'best_price4_len': best_price4_len,
                    'best_price5_len': best_price5_len,
                    'max_total': max_total-test['price'].sum()})            
result_df5 = pd.DataFrame(result_list)
'''

"\nresult_list=[]\nfor cl in df.cluster.unique():\n    test1=df[df.cluster==cl]\n    test=test1[test1.count_price==5]\n    max_total = -np.inf\n    best_price1 = 0\n    best_price2 = 0\n    best_price3 = 0\n    best_price4 = 0\n    best_price5 = 0\n    best_price1_per = 0\n    best_price2_per = 0\n    best_price3_per = 0\n    best_price4_per = 0\n    best_price5_per = 0\n    best_price1_len = 0\n    best_price2_len = 0\n    best_price3_len = 0\n    best_price4_len = 0\n    best_price5_len = 0\n    if test.empty==False:\n        for price1 in range(int(test['price1'].mean()-20000), int(test['price1'].mean()+20000), 10000):\n            for price2 in range(int(test['price2'].mean()-20000), int(price1), 10000):\n                for price3 in range(int(test['price3'].mean()-20000), int(price2), 10000):\n                    for price4 in range(int(test['price4'].mean()-20000), int(price3), 10000):\n                        for price5 in range(int(test['price5'].mean()-20000), int(price4), 10